In [1]:
# [논문용 요약] 6개 요인 집단별 평균 + 상위 3개 요인 + 광고 개수 평균

import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# 1. 데이터 불러오기
filename = "exp - 설문지 응답 시트1 (2).csv"  # 필요하면 파일명만 바꿔서 사용

try:
    df = pd.read_csv(filename)
    print(f"✅ 파일을 성공적으로 읽었습니다! 총 {len(df)}명의 데이터가 있습니다.")
except FileNotFoundError:
    raise FileNotFoundError(f"❌ 파일을 찾을 수 없습니다: {filename}")

# 2. 나이 → 연령대 그룹(20대 초반 / 20대 중후반 / 30대 이상)
age_col = [c for c in df.columns if "[1-2]" in c][0]
df[age_col] = pd.to_numeric(df[age_col].astype(str).str.extract(r"(\d+)")[0], errors="coerce")


def get_age_group(age):
    if pd.isna(age):
        return np.nan
    age = int(age)
    if age <= 25:
        return "20대 초반"
    elif 26 <= age <= 29:
        return "20대 중후반"
    elif age >= 30:
        return "30대 이상"
    else:
        return "기타"


df["Age_Group"] = df[age_col].apply(get_age_group)
print("\n=== 연령대별 표본 수 ===")
print(df["Age_Group"].value_counts())
print("\n" + "="*50)

# 3. 설문 요인 정의를 위한 유틸리티
def find_cols(prefix_list):
    cols = df.columns.tolist()
    found = []
    for prefix in prefix_list:
        matched = [c for c in cols if f"[{prefix}]" in c]
        found.extend(matched)
    return found

# 3-1. SNS 이용률 (횟수/시간 → 숫자)
sns_freq_col = find_cols(["2-3"])[0]
sns_time_col = find_cols(["2-4"])[0]

sns_freq_mapping = {
    "필요시 검색": 1,
    "가끔씩 (2~5회)": 2,
    "수시로 접속 (10회 이상)": 3,
    "거의 습관적으로 자주 (30회 이상)": 4,
    "하루 종일 접속해놓고 있다": 5,
}

sns_time_mapping = {
    "10분 미만": 0.5,
    "30분~1시간 미만": 0.75,
    "1~2시간 미만": 1.5,
    "2~4시간 미만": 3,
    "4~5시간 미만": 4.5,
    "5시간 이상": 6,
}

df["SNS_Freq_Numeric"] = df[sns_freq_col].map(sns_freq_mapping)
df["SNS_Time_Numeric"] = df[sns_time_col].map(sns_time_mapping)

# 3-2. 설문 6개 요인 정의 (스트림릿/코랩에서 쓰던 정의와 동일)
sector_map = {
    "SNS 이용률": ["SNS_Freq_Numeric", "SNS_Time_Numeric"],
    "충동구매 성향": find_cols(["3-1", "3-2", "3-3", "3-4", "3-5", "3-28", "3-29"]),
    "사회적 비교": find_cols(["3-7", "3-8", "3-9", "3-10", "3-11"]),
    "쇼핑 후회/태도": find_cols(["3-13", "3-14", "3-15", "3-16", "3-25", "3-26", "3-27"]),
    "광고 인식/신뢰": find_cols([f"3-{i}" for i in range(17, 25)] + [f"4-{i}" for i in range(1, 13)]),
    "구매 의도": find_cols([f"4-{i}" for i in range(13, 28)]),
}

factor_to_col = {}
for name, cols in sector_map.items():
    if not cols:
        print(f"⚠️ '{name}'에 해당하는 문항을 찾지 못했습니다.")
        continue
    col_name = f"{name}_Mean"
    df[col_name] = df[cols].mean(axis=1)
    factor_to_col[name] = col_name

print("✅ 6개 요인(＋SNS 이용률) 평균 점수 계산 완료")

# 4. 연령대별 6개 요인 평균 (SNS 이용률 제외)
analysis_factors = [name for name in factor_to_col.keys() if name != "SNS 이용률"]
analysis_cols = [factor_to_col[name] for name in analysis_factors]

group_means = df.groupby("Age_Group")[analysis_cols].mean().round(2)

print("\n=== [1] 연령대별 6개 요인 평균 (SNS 이용률 제외) ===")
print(group_means)
print("\n" + "="*50)

# 5. 각 연령대에서 상위 3개 요인 + 전체 순위 (SNS 이용률 제외)
print("\n=== [2] 각 연령대별 상위 3개 요인 (SNS 이용률 제외) 및 전체 순위 ===")

for group in group_means.index:
    row = group_means.loc[group]
    # 컬럼명 → 요인 이름으로 치환
    renamed = row.rename(index={factor_to_col[name]: name for name in analysis_factors})
    sorted_row = renamed.sort_values(ascending=False)
    top3 = sorted_row.head(3)

    print(f"\n■ {group}")
    for i, (fname, val) in enumerate(top3.items(), start=1):
        print(f"  {i}) {fname}: {val:.2f}")

    print("  - 전체 6개 요인 순위 (내림차순):")
    for fname, val in sorted_row.items():
        print(f"    · {fname}: {val:.2f}")

print("\n" + "="*50)

# 6. 연령대별 광고 개수 평균 ([2-6] 문항)

ad_cols = [c for c in df.columns if "[2-6]" in c]
if ad_cols:
    ad_col = ad_cols[0]
    df["Ad_Count_Numeric"] = pd.to_numeric(df[ad_col], errors="coerce")
    ad_means = df.groupby("Age_Group")["Ad_Count_Numeric"].mean().round(2)
    print("\n=== [3] 연령대별 광고 개수 평균 ([2-6]) ===")
    print(ad_means)
else:
    print("\n⚠️ [2-6] 문항(광고 개수)을 찾을 수 없습니다.")

✅ 파일을 성공적으로 읽었습니다! 총 122명의 데이터가 있습니다.

=== 연령대별 표본 수 ===
Age_Group
20대 초반     52
30대 이상     30
20대 중후반    29
Name: count, dtype: int64

✅ 6개 요인(＋SNS 이용률) 평균 점수 계산 완료

=== [1] 연령대별 6개 요인 평균 (SNS 이용률 제외) ===
           충동구매 성향_Mean  사회적 비교_Mean  쇼핑 후회/태도_Mean  광고 인식/신뢰_Mean  구매 의도_Mean
Age_Group                                                                     
20대 중후반            2.14         2.86           3.22           2.79        2.60
20대 초반             2.47         3.07           3.24           2.79        2.61
30대 이상             2.19         2.73           3.02           2.70        2.45


=== [2] 각 연령대별 상위 3개 요인 (SNS 이용률 제외) 및 전체 순위 ===

■ 20대 중후반
  1) 쇼핑 후회/태도: 3.22
  2) 사회적 비교: 2.86
  3) 광고 인식/신뢰: 2.79
  - 전체 6개 요인 순위 (내림차순):
    · 쇼핑 후회/태도: 3.22
    · 사회적 비교: 2.86
    · 광고 인식/신뢰: 2.79
    · 구매 의도: 2.60
    · 충동구매 성향: 2.14

■ 20대 초반
  1) 쇼핑 후회/태도: 3.24
  2) 사회적 비교: 3.07
  3) 광고 인식/신뢰: 2.79
  - 전체 6개 요인 순위 (내림차순):
    · 쇼핑 후회/태도: 3.24
    · 사회적 비교: 3.07
    · 광고 인식/신뢰: 2.79
   